In [ ]:
import numpy as np
from scipy.optimize import minimize

def portfolio_statistics(weights, mean_returns, cov_matrix, risk_free_rate=0.01):
    """
    Calculate portfolio return, risk (volatility), and Sharpe ratio.

    Args:
        weights (np.array): Portfolio weights.
        mean_returns (np.array): Mean daily returns of stocks.
        cov_matrix (np.array): Covariance matrix of returns.
        risk_free_rate (float): Risk-free rate (default: 1% annualized).

    Returns:
        tuple: Portfolio return, portfolio volatility, Sharpe ratio.
    """
    weights = np.array(weights)
    portfolio_return = np.dot(weights, mean_returns) * 252  # Annualize returns
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)  # Annualize volatility
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility

    return portfolio_return, portfolio_volatility, sharpe_ratio


def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    """
    Objective function to maximize Sharpe ratio (negative for minimization).
    """
    return -portfolio_statistics(weights, mean_returns, cov_matrix, risk_free_rate)[2]


def minimize_volatility(weights, mean_returns, cov_matrix):
    """
    Objective function to minimize portfolio volatility.
    """
    return portfolio_statistics(weights, mean_returns, cov_matrix)[1]


def optimize_portfolio(mean_returns, cov_matrix, risk_free_rate=0.01, optimization_goal="sharpe"):
    """
    Optimize portfolio weights to minimize risk or maximize Sharpe ratio.

    Args:
        mean_returns (np.array): Mean daily returns of stocks.
        cov_matrix (np.array): Covariance matrix of returns.
        risk_free_rate (float): Risk-free rate (default: 1% annualized).
        optimization_goal (str): Optimization goal ("sharpe" or "volatility").

    Returns:
        dict: Optimized weights, expected return, risk, and Sharpe ratio.
    """
    num_assets = len(mean_returns)
    initial_weights = np.ones(num_assets) / num_assets
    bounds = tuple((0, 1) for _ in range(num_assets))  # No short selling
    constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}  # Weights sum to 1

    if optimization_goal == "sharpe":
        objective_function = lambda w: negative_sharpe_ratio(w, mean_returns, cov_matrix, risk_free_rate)
    elif optimization_goal == "volatility":
        objective_function = lambda w: minimize_volatility(w, mean_returns, cov_matrix)

    optimized = minimize(objective_function, initial_weights, method="SLSQP", bounds=bounds, constraints=constraints)

    if not optimized.success:
        raise ValueError("Optimization failed!")

    optimized_weights = optimized.x
    return_stats = portfolio_statistics(optimized_weights, mean_returns, cov_matrix, risk_free_rate)
    
    return {
        "weights": optimized_weights,
        "expected_return": return_stats[0],
        "volatility": return_stats[1],
        "sharpe_ratio": return_stats[2]
    }
